In [17]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt

# For data augmentation and pipeline handling
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [18]:
# Example: Set up an ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(
    
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # Reserve 20% for validation
)

# Update the paths to point to your training directory
train_dir = "C:/Users/5A_Traders/Downloads/FYP_ON_DEV/FYP_IntelliTrain/Datasets/pose/MyData"

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(192, 192),  # Resize images to the input size expected by MoveNet
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(192, 192),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 30 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [19]:
# Load MoveNet model from TF Hub (singlepose lightning version)
movenet = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")

def extract_pose_keypoints(image):
    """
    Extracts pose keypoints using MoveNet.
    
    Args:
        image: A single image tensor of shape (192, 192, 3).
    
    Returns:
        A flattened vector of keypoints.
    """
    # Add batch dimension and ensure image is float32
    image = tf.cast(image * 255.0, tf.int32)
    input_image = tf.expand_dims(image, axis=0)
    #input_image = tf.expand_dims(tf.cast(image, tf.float32), axis=0)
    # Run inference
    outputs = movenet.signatures['serving_default'](input_image)
    keypoints = outputs['output_0']
    # Reshape keypoints to a 1D tensor (for 17 keypoints with 3 values each: y, x, score)
    keypoints = tf.reshape(keypoints, [-1])
    return keypoints


In [20]:
from tensorflow.keras import layers, models

# Define a custom layer to extract pose keypoints from an input image
def PoseKeypointLayer(x):
    # x is a batch of images of shape (batch, 192, 192, 3)
    keypoints = tf.map_fn(extract_pose_keypoints, x, dtype=tf.float32)
    return keypoints

# Build the model
input_img = layers.Input(shape=(192, 192, 3), name='input_image')

# Lambda layer to extract pose keypoints; this layer is non-trainable
keypoints = layers.Lambda(PoseKeypointLayer, name='pose_extraction')(input_img)

# Add a couple of dense layers for classification
x = layers.Dense(128, activation='relu')(keypoints)
x = layers.Dropout(0.5)(x)  # Dropout for regularization
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.3)(x)
# Adjust the number of output units to match your number of classes, e.g., 3
output = layers.Dense(train_generator.num_classes, activation='softmax', name='output')(x)

model = models.Model(inputs=input_img, outputs=output, name='Pose_Estimation_Classifier')
model.summary()


Model: "Pose_Estimation_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_image (InputLayer)             │ (None, 192, 192, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ pose_extraction (Lambda)             │ (None, 51)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │           6,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ output (Dense)                       │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,042 (58.76 KB)

 Trainable params: 15,042 (58.76 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    callbacks=callbacks
)


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 52s 52s/step - accuracy: 0.3333 - loss: 0.7554 - val_accuracy: 0.6667 - val_loss: 0.6907 - learning_rate: 0.0010
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4333 - loss: 0.7114 - val_accuracy: 0.5000 - val_loss: 0.6940 - learning_rate: 0.0010
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.4667 - loss: 0.7258 - val_accuracy: 0.5000 - val_loss: 0.7043 - learning_rate: 0.0010
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.7144 - learning_rate: 0.0010
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4000 - loss: 0.7630 - val_accuracy: 0.5000 - val_loss: 0.7022 - learning_rate: 5.0000e-04
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4667 - loss: 0.7052 - val_accuracy: 0.5000 - val_loss: 0.7161 - learning_rate: 5.0000e-04


In [22]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Save the model
model.save('pose_estimation_teachable_machine_clone.h5')


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.6967


Validation Loss: 0.6967, Validation Accuracy: 0.5000


In [29]:
# Define the target image size (width, height)
image_size = (192, 192)

def predict_image(image_path):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    # Load the image and resize it to the target size
    img = image.load_img(image_path, target_size=image_size)
    img_array = image.img_to_array(img)
    
    # Convert the image to int32 to meet the model's requirements
    img_array = img_array.astype('int32')
    
    # Expand dimensions to create a batch of size 1
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict using the trained model
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)
    
    # Return the class mapping, predicted class index, and prediction confidence
    return train_generator.class_indices, predicted_class, confidence

# Example usage
class_indices, predicted_class, confidence = predict_image("C:/Users/5A_Traders/Pictures/Camera Roll/WIN_20250224_09_09_19_Pro.jpg")
print("Class Indices:", class_indices)
print("Predicted Class:", predicted_class)
print("Confidence:", confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 688ms/step
Class Indices: {'Hand Down': 0, 'Hand Up': 1}
Predicted Class: 0
Confidence: 0.5000483


NameError: name 'image_size' is not defined